<font size = "22">OBJECTIVES</font>

<font size = "5"> Our goal is to create a machine learning model to predict which indviduals are most likely to have or use a commercial bank account. The metric used is the mean absolute error. To know more about our dataset, kindly refer to the readme file.</font>

<font size = "22"> MY APPROACH </font> 

<font size = "5"> To build a machine model that minimizes our mean absolute error, I used an ensemble containing different machine learning algorithms namely XGBOOST, LIGHTGBM and CATBOOST. </font>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<font size = "22"> LIBRARIES </FONT>

In [1]:
#Import libraries
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
import pandas as pd 
import numpy as np 
from sklearn.utils import shuffle
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error
from catboost import Pool

<font size = "22"> LOAD DATASETS </font>

In [2]:
train = pd.read_csv('drive/MyDrive/financial-inclusion-in-africa/Train.csv')
test = pd.read_csv('drive/MyDrive/financial-inclusion-in-africa/Test.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [3]:
train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


<font size = "22"> DATA PREPROCESSING AND ANALYSIS </font>

<font size = "5"> This section is divided into 2 parts. First, we are going to preprocess our data for xgboost and catboost models then we are going to preprocess our data for lightgbm

In [5]:
#We create an ID feature for submission
id = test['uniqueid'] + ' x ' + test['country']

In [6]:
#We drop the unique id column in our train and test datasets
train.drop('uniqueid', inplace = True, axis = 1)
test.drop('uniqueid', inplace = True, axis = 1)

In [7]:
#We shuffle our train dataset
train = shuffle(train, random_state = 2)

In [8]:
train['age_range'] = pd.cut(train['age_of_respondent'], bins = [0, 12, 19, 40, 60, 100], labels = ['Child', 'Teenager', 'Young Adult', 'Middle Aged', 'Old'])
test['age_range'] = pd.cut(test['age_of_respondent'], bins = [0, 12, 19, 40, 60, 100], labels = ['Child', 'Teenager', 'Young Adult', 'Middle Aged', 'Old'])
train.drop('age_of_respondent',axis = 1, inplace = True)
test.drop('age_of_respondent', axis = 1, inplace = True)

In [9]:
train['bank_account'] = np.where(train['bank_account'] == 'No', 0, 1)
train['bank_account']

7677     0
4403     1
9395     0
10271    0
14213    0
        ..
1099     0
18898    0
11798    0
6637     0
2575     1
Name: bank_account, Length: 23524, dtype: int64

In [10]:
#We create a copy of our datasets, we are going to use this dataset to train our lightgbm model 
train_copy = train.copy()
test_copy = test.copy()

In [11]:
x_train, y_train = train.drop('bank_account', axis = 1), train['bank_account']

In [12]:
#Add our test dataset to our train dataset
total_df = pd.concat([x_train, test], axis = 0)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33610 entries, 7677 to 10085
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   country                 33610 non-null  object  
 1   year                    33610 non-null  int64   
 2   location_type           33610 non-null  object  
 3   cellphone_access        33610 non-null  object  
 4   household_size          33610 non-null  int64   
 5   gender_of_respondent    33610 non-null  object  
 6   relationship_with_head  33610 non-null  object  
 7   marital_status          33610 non-null  object  
 8   education_level         33610 non-null  object  
 9   job_type                33610 non-null  object  
 10  age_range               33610 non-null  category
dtypes: category(1), int64(2), object(8)
memory usage: 2.9+ MB


In [13]:
#We convert our age_range feature from categorical to object
total_df['age_range'] = total_df['age_range'].astype(object)

In [14]:
#Perform one hot encoding
total_df = pd.get_dummies(total_df, columns = ['country', 'location_type', 'cellphone_access', 'gender_of_respondent','relationship_with_head', 'marital_status',
                                                        'education_level', 'job_type', 'age_range'])

In [15]:
#Normalize our data
total_df[['household_size', 'year']] = np.log1p(total_df[['household_size', 'year']])

In [16]:
#We then split our dataset into train and test datasets
x_train_pro, test_pro = total_df.iloc[:23524,:], total_df.iloc[23524:, :]
x_train_pro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23524 entries, 7677 to 2575
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   year                                             23524 non-null  float64
 1   household_size                                   23524 non-null  float64
 2   country_Kenya                                    23524 non-null  uint8  
 3   country_Rwanda                                   23524 non-null  uint8  
 4   country_Tanzania                                 23524 non-null  uint8  
 5   country_Uganda                                   23524 non-null  uint8  
 6   location_type_Rural                              23524 non-null  uint8  
 7   location_type_Urban                              23524 non-null  uint8  
 8   cellphone_access_No                              23524 non-null  uint8  
 9   cellphone_access_Yes      

In [17]:
#Create a validation dataset
x_valid, x_train, y_valid, y_train = x_train_pro[:500], x_train_pro[500:], y_train[:500], y_train[500:]

In [18]:
#Create a validation dataset for our lightgbm model
valid, train_copy = train_copy[:500], train_copy[500:]

In [19]:
#Split our dataset into training data and labels
x_train_copy, y_train_copy = train_copy.drop('bank_account', axis = 1), train_copy['bank_account']
x_valid_copy, y_valid_copy = valid.drop('bank_account', axis = 1), valid['bank_account']

In [20]:
#Convert categorical features from object datatype to category datatype
#This is important as lightgbm works well with categorical features
object_list = [x for x in x_train_copy.columns if x_train_copy[x].dtype == object]
x_train_copy[object_list] = x_train_copy[object_list].astype('category')
x_valid_copy[object_list] = x_valid_copy[object_list].astype('category')
test_copy[object_list] = test_copy[object_list].astype('category')

In [21]:
#Normalize our training and validation datasets
int_list = [x for x in x_train_copy.columns if x_train_copy[x].dtype == 'int64']
x_train_copy[int_list] = np.log1p(x_train_copy[int_list])
x_valid_copy[int_list] = np.log1p(x_valid_copy[int_list])

In [22]:
#Normalize our test dataset
test_copy[int_list] = np.log1p(test_copy[int_list])

<font size = "22"> MODELLING </font>

In [23]:
#Train our xgboost classifier
classifier = XGBClassifier(learning_rate=0.2, n_estimators = 400)
classifier.fit(x_train, y_train)

XGBClassifier(learning_rate=0.2, n_estimators=400)

In [24]:
#Train our catboost classifier
classifier_cat = CatBoostClassifier(n_estimators = 1000)
classifier_cat.fit(x_train, y_train)

Learning rate set to 0.039317
0:	learn: 0.6517074	total: 55.8ms	remaining: 55.8s
1:	learn: 0.6203047	total: 62.4ms	remaining: 31.1s
2:	learn: 0.5914446	total: 68.9ms	remaining: 22.9s
3:	learn: 0.5608907	total: 75.5ms	remaining: 18.8s
4:	learn: 0.5370651	total: 81.3ms	remaining: 16.2s
5:	learn: 0.5178305	total: 87.7ms	remaining: 14.5s
6:	learn: 0.4996303	total: 94.2ms	remaining: 13.4s
7:	learn: 0.4825999	total: 101ms	remaining: 12.5s
8:	learn: 0.4678810	total: 107ms	remaining: 11.8s
9:	learn: 0.4517892	total: 114ms	remaining: 11.3s
10:	learn: 0.4362494	total: 121ms	remaining: 10.9s
11:	learn: 0.4257607	total: 128ms	remaining: 10.5s
12:	learn: 0.4152350	total: 134ms	remaining: 10.2s
13:	learn: 0.4050876	total: 141ms	remaining: 9.93s
14:	learn: 0.3963527	total: 148ms	remaining: 9.69s
15:	learn: 0.3897932	total: 154ms	remaining: 9.47s
16:	learn: 0.3829047	total: 161ms	remaining: 9.28s
17:	learn: 0.3758907	total: 167ms	remaining: 9.12s
18:	learn: 0.3698359	total: 174ms	remaining: 8.99s
19:	

In [25]:
#Train our lightgbm classifier
classifier_light = LGBMClassifier(max_depth=7, num_leaves=18, n_estimators=90, loss_fuction='mean_absolute_error')
classifier_light.fit(x_train_copy, y_train_copy)

LGBMClassifier(loss_fuction='mean_absolute_error', max_depth=7, n_estimators=90,
               num_leaves=18)

In [26]:
#We create an ensemble of our models and make predictions on our validation dataset
pred_final1 = classifier.predict(x_valid)
pred_final2 = classifier_light.predict(x_valid_copy)
pred_final3 = classifier_cat.predict(x_valid)
predBlend = (pred_final1 + pred_final2 + pred_final3) / 6
preds = [1 if x >= 0.5 else 0 for x in predBlend]

In [27]:
error = mean_absolute_error(y_valid, preds)
error

0.092

In [28]:
#We make predictions on our test data
pred1 = classifier.predict(test_pro)
pred2 = classifier_light.predict(test_copy)
pred3 = classifier_cat.predict(test_pro)
pred4 = (pred1 + pred2 + pred3) / 6
pred5 = [1 if x >= 0.5 else 0 for x in pred4]

In [29]:
dic = {'unique_id': id, 'bank_account': pred5}
pre1 = pd.DataFrame(dic)
pre1.to_csv('fin_com.csv', index = False)